In [40]:
import pandas as pd
from sentence_transformers import SentenceTransformer
import ast

## Merge json file together

In [5]:
res = pd.DataFrame()

In [7]:
import os

files = os.listdir('data/train')
index = 0
num_of_jsons = 10

# 10 json files atm
for i in range(num_of_jsons):
    filename = files[i]
    print(filename)
    df = pd.read_json(f'data/train/{filename}')
    df = df.drop(['tokens', 'diffbotUri', 'text', 'images'], axis=1)

    model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

    df['keys'] = ''
    df['values'] = ''
    df['title embeddings'] = ''

    i = 0
    for row in df['specs']:
        row_keys = list(row.keys())
        row_values = list(row.values())
        df.at[i, 'keys'] = row_keys
        df.at[i, 'values'] = row_values
        i += 1

    a = 0
    for row in df['title']:
        title_embed = model.encode(row)
        df.at[a, 'title embeddings'] = title_embed
        a += 1


    df = df.drop('specs', axis=1)
    res = res.append(df)
    print(f'df shape{i}:{df.shape}')
    print(f'res shape{i}:{res.shape}')
    index += 1

dataset-000.json
df shape2173:(2173, 4)
res shape2173:(2173, 4)
dataset-001.json
df shape2351:(2351, 4)
res shape2351:(4524, 4)
dataset-002.json
df shape2281:(2281, 4)
res shape2281:(6805, 4)
dataset-003.json
df shape2260:(2260, 4)
res shape2260:(9065, 4)
dataset-004.json
df shape2314:(2314, 4)
res shape2314:(11379, 4)
dataset-005.json
df shape2161:(2161, 4)
res shape2161:(13540, 4)
dataset-006.json
df shape2332:(2332, 4)
res shape2332:(15872, 4)
dataset-007.json
df shape2224:(2224, 4)
res shape2224:(18096, 4)
dataset-008.json
df shape2305:(2305, 4)
res shape2305:(20401, 4)
dataset-009.json
df shape2264:(2264, 4)
res shape2264:(22665, 4)


In [8]:
res = res.drop_duplicates(subset='title')
res.shape
res.to_pickle(f'{num_of_jsons}jsons.pkl')

## Cluster with DBSCAN

In [3]:
res = pd.read_pickle(r'10jsons.pkl')

In [4]:
from sklearn.cluster import DBSCAN

final = pd.DataFrame()
outliers = pd.DataFrame()
epsilon = [6, 5, 4, 3, 2, 1, 0.5]

# Cluster title embeddings (combined 10 MAE data parts)

num_out = len(res)
i = 0

if i == 0:
    print(i, f'epsilon = {epsilon[i]}')
    title_embeds = []

    for row in res['title embeddings']:
        title_embeds.append(row)

    db = DBSCAN(eps=epsilon[0], min_samples=10).fit(title_embeds)
    title_labels = db.labels_

    titles = res['title']
    keys = res['keys']
    title_embed = res['title embeddings']
    df_new = list(zip(titles, keys,  title_labels, title_embed))
    sorted_titles = pd.DataFrame(df_new, columns=['titles', 'keys', 'cluster_num', 'title_embeds'])
    sorted_titles = sorted_titles.sort_values(by=['cluster_num'])
    outliers = sorted_titles[sorted_titles.cluster_num <= 0]
    print(f'outliers shape: {outliers.shape}')
    clusters = sorted_titles[sorted_titles.cluster_num > 0]
    print(f'clusters shape: {clusters.shape}')
    #sorted_titles.to_csv('clusters/sorted_titles_mae10.csv')
    final = final.append(clusters)
    print(f'final shape: {final.shape}')


    # Number of clusters in labels, ignoring noise if present.
    n_clusters_ = len(set(title_labels)) - (1 if -1 in title_labels else 0)
    n_noise_ = list(title_labels).count(-1)

    print("Estimated number of title clusters: %d" % n_clusters_)
    print("Estimated number of title noise points: %d" % n_noise_)

    i += 1

if i != 0:
    
    while i < len(epsilon):
        print(i, f'epsilon = {epsilon[i]}')
        title_embeds = []

        for row in outliers['title_embeds']:
            title_embeds.append(row)

        db = DBSCAN(eps=epsilon[i], min_samples=5).fit(title_embeds)
        title_labels = db.labels_

        titles = outliers['titles']
        keys = outliers['keys']
        title_embed = outliers['title_embeds']
        df_new = list(zip(titles, keys,  title_labels, title_embed))
        sorted_titles = pd.DataFrame(df_new, columns=['titles', 'keys', 'cluster_num', 'title_embeds'])
        sorted_titles = sorted_titles.sort_values(by=['cluster_num'])
        outliers = sorted_titles[sorted_titles.cluster_num <= 0]
        print(f'outliers shape: {outliers.shape}')
        clusters = sorted_titles[sorted_titles.cluster_num > 0]
        print(f'clusters shape: {clusters.shape}')
        #sorted_titles.to_csv('clusters/sorted_titles_mae10.csv')
        final = final.append(clusters)
        print(f'final shape: {final.shape}')



        # Number of clusters in labels, ignoring noise if present.
        n_clusters_ = len(set(title_labels)) - (1 if -1 in title_labels else 0)
        n_noise_ = list(title_labels).count(-1)

        print("Estimated number of title clusters: %d" % n_clusters_)
        print("Estimated number of title noise points: %d" % n_noise_)

        i += 1
        
'''        if i == 2:
            break
        i += 1'''

0 epsilon = 6
outliers shape: (19210, 4)
clusters shape: (0, 4)
final shape: (0, 4)
Estimated number of title clusters: 1
Estimated number of title noise points: 4557
1 epsilon = 5
outliers shape: (11461, 4)
clusters shape: (7749, 4)
final shape: (7749, 4)
Estimated number of title clusters: 46
Estimated number of title noise points: 11456
2 epsilon = 4
outliers shape: (11461, 4)
clusters shape: (0, 4)
final shape: (7749, 4)
Estimated number of title clusters: 0
Estimated number of title noise points: 11461
3 epsilon = 3
outliers shape: (11461, 4)
clusters shape: (0, 4)
final shape: (7749, 4)
Estimated number of title clusters: 0
Estimated number of title noise points: 11461
4 epsilon = 2
outliers shape: (11461, 4)
clusters shape: (0, 4)
final shape: (7749, 4)
Estimated number of title clusters: 0
Estimated number of title noise points: 11461
5 epsilon = 1
outliers shape: (11461, 4)
clusters shape: (0, 4)
final shape: (7749, 4)
Estimated number of title clusters: 0
Estimated number of 

'        if i == 2:\n            break\n        i += 1'

In [49]:
end = final.drop(['title_embeds'], axis=1)
end.to_csv('phase1.csv')

## Attribute Domain

Find unique attributes and loop through items

If item has the unique attribute add this item to the cluster

In [5]:
unique_keys = final
unique_keys = unique_keys.explode('keys')
unique_keys.shape

(35450, 4)

In [54]:
#unique_keys = unique_keys.sort_values()
unique_keys = unique_keys.drop_duplicates('keys')
unique_keys

,titles,keys,cluster_num,title_embeds
3379,Maserati GranTurismo Prices & Deals - Texas,transmissions_available,1,"[-0.28324303, -0.11341931, 0.01945446, 0.53519..."
3379,Maserati GranTurismo Prices & Deals - Texas,seating,1,"[-0.28324303, -0.11341931, 0.01945446, 0.53519..."
3379,Maserati GranTurismo Prices & Deals - Texas,drive_wheels,1,"[-0.28324303, -0.11341931, 0.01945446, 0.53519..."
16350,HP Inc. EliteDesk 800 G2 - Mini desktop - 1 x ...,graphics,2,"[-0.07585266, 0.1031892, 0.05065245, -0.048024..."
16350,HP Inc. EliteDesk 800 G2 - Mini desktop - 1 x ...,storage_type,2,"[-0.07585266, 0.1031892, 0.05065245, -0.048024..."
...,...,...,...,...
12174,2017 wholesale! New Fashion Australia classic ...,shoe_width,37,"[-0.7255332, 0.28313932, 0.03292244, 0.3220458..."
12460,Bonnibel GF68 Women's Faux Suede Lace-up Insid...,toe_shape,37,"[-0.66307425, 0.044928115, 0.14649808, -0.0040..."
7656,The Book of Greens: A Cook's Compendium of 40 ...,hardcover,43,"[-0.032763556, -0.09156644, -0.23731938, 0.396..."
18273,Cook Lively!: 100 Quick and Easy Plant-Based R...,paperback,43,"[-0.18450098, 0.13388027, 0.303892, 0.29558322..."


Maybe cluster keys first?

In [7]:
# if a key is in an item place that item in the cluster that correspondes with the key

new_clusters = clusters
new_clusters.shape
new_outliers = outliers
new_outliers.shape

(11461, 4)

In [55]:
cluster_keys = unique_keys['keys']


for index, row in unique_keys.iterrows():
    un_key = row['keys']
    cluster_num = row['cluster_num']
    print(f'Unique key: {un_key}')
    print(f'cluster number: {cluster_num}')
    for index_out, row_out in new_outliers.iterrows():
        keys = row_out['keys']
        item = row_out['titles']
        if un_key in keys:
            # change to matching custer number
            outliers.at[index_out, 'cluster_num'] = cluster_num
            # add item to cluster
            new_clusters = new_clusters.append(row_out)
            # remove item from outliers
            new_outliers = new_outliers.drop(index_out, axis=0)

Unique key: transmissions_available
cluster number: 1
Unique key: seating
cluster number: 1
Unique key: drive_wheels
cluster number: 1
Unique key: graphics
cluster number: 2
Unique key: storage_type
cluster number: 2
Unique key: chipset
cluster number: 2
Unique key: memory_slots
cluster number: 2
Unique key: form_factor
cluster number: 2
Unique key: keyboard
cluster number: 2
Unique key: warranty
cluster number: 2
Unique key: number_of_speakers
cluster number: 2
Unique key: connection
cluster number: 2
Unique key: bluetooth_version
cluster number: 2
Unique key: unit_type
cluster number: 2
Unique key: mtbf
cluster number: 2
Unique key: interface
cluster number: 2
Unique key: refillable
cluster number: 2
Unique key: manufacturer
cluster number: 2
Unique key: packaged_quantity
cluster number: 2
Unique key: features
cluster number: 2
Unique key: global_product_type
cluster number: 2
Unique key: number_of_plies
cluster number: 2
Unique key: packaging
cluster number: 2
Unique key: scent
clus

In [9]:
print(f'new outliers shape: {new_outliers.shape}')
print(f'new clusters shape: {new_clusters.shape}')

new outliers shape: (616, 4)
new clusters shape: (10845, 4)


In [10]:
new_clusters = new_clusters.sort_values(by=['cluster_num'])

In [50]:
new_clusters = new_clusters.drop(['title_embeds'], axis=1)
new_clusters.to_csv('phase2.csv')

## Attribute sets
Find clusters of attribute groups

If items have the same attribute group this is a new cluster

In [23]:
groups = new_outliers.sort_values(by=['keys'])

,titles,keys,cluster_num,title_embeds
3493,Fallout Vintage Nuka Cola T-Shirt,[3t],-1,"[-0.31513876, 0.47561297, 0.08851613, 0.280716..."
9667,Long-Sleeve Geo Print Heart Graphic Tee,"[age_of_oldest_child, age_of_youngest_child]",-1,"[-0.013641296, 0.24473599, 0.15723602, -0.0018..."
10437,Zip-Up Heavyweight Fleece Jacket,"[age_of_oldest_child, age_of_youngest_child]",-1,"[-0.54284215, 1.146121, -0.4064959, 0.3546009,..."
10274,You May Also Like,"[age_of_oldest_child, age_of_youngest_child]",-1,"[0.111564785, -0.6545003, 0.6776573, -0.216990..."
8960,2-Pack Little Rocket Babysoft Sleeper Gown Set,"[age_of_oldest_child, number_of_kids]",-1,"[0.21879672, 0.8875228, -0.0035157616, 0.55963..."
...,...,...,...,...
7041,Hostellerie Van Gaalen,"[year_last_renovated, check_out_time]",-1,"[0.04361704, 0.7176301, -0.08374879, -0.069743..."
10549,Hotel Laguna Istra,"[year_last_renovated, check_out_time, check_in...",-1,"[0.87141603, -0.20207971, -0.018911682, 0.4232..."
8129,McClelland-Priest Bed & Breakfast Inn,"[year_last_renovated, check_out_time, check_in...",-1,"[0.0079735285, -0.20539731, -0.12590931, 0.193..."
6961,Oriental Silk Hotel Guangzhou,"[year_last_renovated, check_out_time, check_in...",-1,"[-0.0554308, 0.39309227, 0.3962692, 0.53304815..."


In [30]:
# get all key groups that belong to at least 10 items

min_items = 10

df = groups.astype(str)
df_out = df.groupby(['keys']).size().reset_index(name='counter')

cluster_key_list = []

for index_g, row_g in df_out.iterrows():
    count = row_g['counter']
    keys_group = row_g['keys']
    if count >= min_items:
        cluster_key_list.append(keys_group)

print(cluster_key_list)

["['age_of_oldest_child', 'number_of_kids', 'age_of_youngest_child']", "['browse_similar_products_by_category']", "['copyright', 'copyright_holder']", "['discs']", "['international_shipping']", "['measurements']", "['orig_year', 'discs']", "['pages']", "['sm', 'waist', 'us']", "['x-large']"]


In [46]:
att_sets = pd.DataFrame()

cluster_number = 1

for item in cluster_key_list:
    item = ast.literal_eval(item)
    item = [i.strip() for i in item]
    print(item)
    for index_g, row_g in groups.iterrows():
        key_group = row_g['keys']
        if item == key_group:
            # change to matching custer number
            groups.at[index_g, 'cluster_num'] = cluster_number
            print(row_g)
            # add item to cluster
            att_sets = att_sets.append(row_g)
    cluster_number += 1
            
att_sets


['age_of_oldest_child', 'number_of_kids', 'age_of_youngest_child']
titles                      2-Piece Snug Fit Cotton Christmas PJs
keys            [age_of_oldest_child, number_of_kids, age_of_y...
cluster_num                                                     1
title_embeds    [-0.6713714, 0.36546147, -0.22957157, 0.325324...
Name: 9211, dtype: object
titles                                  5-Pack Tank Top Bodysuits
keys            [age_of_oldest_child, number_of_kids, age_of_y...
cluster_num                                                     1
title_embeds    [0.8743884, 0.024854254, 0.6004976, 0.00790057...
Name: 8506, dtype: object
titles                                  3-Piece Little Jacket Set
keys            [age_of_oldest_child, number_of_kids, age_of_y...
cluster_num                                                     1
title_embeds    [-0.5820369, 0.7727704, -0.14010243, 0.3643328...
Name: 6459, dtype: object
titles                                4-Piece Snug Fit Cotton P

,titles,keys,cluster_num,title_embeds
9211,2-Piece Snug Fit Cotton Christmas PJs,"[age_of_oldest_child, number_of_kids, age_of_y...",1.0,"[-0.6713714, 0.36546147, -0.22957157, 0.325324..."
8506,5-Pack Tank Top Bodysuits,"[age_of_oldest_child, number_of_kids, age_of_y...",1.0,"[0.8743884, 0.024854254, 0.6004976, 0.00790057..."
6459,3-Piece Little Jacket Set,"[age_of_oldest_child, number_of_kids, age_of_y...",1.0,"[-0.5820369, 0.7727704, -0.14010243, 0.3643328..."
450,4-Piece Snug Fit Cotton PJs,"[age_of_oldest_child, number_of_kids, age_of_y...",1.0,"[-0.35220894, 0.092805736, -0.3588692, -0.0247..."
532,Tiered Ruffle Strawberry Print Dress,"[age_of_oldest_child, number_of_kids, age_of_y...",1.0,"[0.14028901, -0.024648342, 0.7096772, 0.396929..."
...,...,...,...,...
5520,(i'm not with him) Dark T-Shirt,[x-large],10.0,"[-0.05715561, -0.0010958378, 0.2849006, -0.128..."
6591,UNCLE RALPH Twill Mens Moto Jogger Pants,[x-large],10.0,"[-0.69565046, 0.7742309, 0.1927521, 0.1278764,..."
10424,Chicago Cubs World Series Champions 3 Pack Gol...,[x-large],10.0,"[0.1712608, 0.21641505, -0.41933894, -0.182341..."
1135,Dewey Beach DE - Varsity Design Dark T-Shirt,[x-large],10.0,"[-0.5227422, -0.0029918153, 0.49394706, -0.019..."


In [53]:
att_sets = att_sets.drop(['title_embeds'], axis=1)
att_sets.to_csv('phase3.csv')